In [1]:
import torch
import sys
import os
import time
import numpy as np
import argparse
sys.path.append("..")

from torch.utils.data import DataLoader
from torch.utils.data import WeightedRandomSampler
from umap.umap_ import find_ab_params

from singleVis.custom_weighted_random_sampler import CustomWeightedRandomSampler
from singleVis.SingleVisualizationModel import VisModel
from singleVis.losses import UmapLoss, ReconstructionLoss, SingleVisLoss
from singleVis.edge_dataset import DataHandler
from singleVis.trainer import SingleVisTrainer
from singleVis.data import NormalDataProvider
from singleVis.spatial_edge_constructor import kcSpatialAlignmentEdgeConstructor
# from singleVis.temporal_edge_constructor import GlobalTemporalEdgeConstructor
from singleVis.alignment_edge_constructor import LocalAlignmentEdgeConstructor
from singleVis.projector import TimeVisProjector
from singleVis.eval.evaluator import Evaluator


import torch
import numpy as np
from CKA import CKA, CudaCKA

# REF_PATH : reference dataset path
# CONFUSION_PATH : benchmark1
# EXCHANGE_PATH : benchmark2

REF_PATH = "/home/yifan/dataset/noisy/pairflip/cifar10/noisy0.001"
CLEAN_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"

CONFUSION_PATH = "/home/yifan/dataset/confusion/pairflip/cifar10/0"
EXCHANGE_PATH = "/home/yifan/dataset/exchange/pairflip/cifar10/0"

sys.path.append(REF_PATH)


from config import config

SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]
GPU_ID = config["GPU"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
PREPROCESS = VISUALIZATION_PARAMETER["PREPROCESS"]
LAMBDA = VISUALIZATION_PARAMETER["LAMBDA"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
INIT_NUM = VISUALIZATION_PARAMETER["INIT_NUM"]
ALPHA = VISUALIZATION_PARAMETER["ALPHA"]
BETA = VISUALIZATION_PARAMETER["BETA"]
MAX_HAUSDORFF = VISUALIZATION_PARAMETER["MAX_HAUSDORFF"]
# HIDDEN_LAYER = VISUALIZATION_PARAMETER["HIDDEN_LAYER"]
ENCODER_DIMS = VISUALIZATION_PARAMETER["ENCODER_DIMS"]
DECODER_DIMS = VISUALIZATION_PARAMETER["DECODER_DIMS"]
S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
T_N_EPOCHS = VISUALIZATION_PARAMETER["T_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]

VIS_MODEL_NAME = 'vis'
EVALUATION_NAME = VISUALIZATION_PARAMETER["EVALUATION_NAME"]

SEGMENTS = [(EPOCH_START, EPOCH_END)]
# define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))


ref_provider = NormalDataProvider(REF_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)
clean_provider = NormalDataProvider(CLEAN_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)

confusion_provider = NormalDataProvider(CONFUSION_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)

exchange_provider = NormalDataProvider(EXCHANGE_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)


ref_train_data = ref_provider.train_representation(200).squeeze()

confusion_data = confusion_provider.train_representation(200).squeeze()

exchange_data = exchange_provider.train_representation(200).squeeze()

clean_data = clean_provider.train_representation(200).squeeze()







/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Finish initialization...
Finish initialization...
Finish initialization...
Finish initialization...


In [4]:
from torch import nn
from torch.autograd import Variable

import torch.nn.functional as F
import scipy
import numpy as np
from sklearn.cross_decomposition import CCA

model = VisModel(ENCODER_DIMS, DECODER_DIMS)
projector = TimeVisProjector(vis_model=model, content_path=REF_PATH, vis_model_name=VIS_MODEL_NAME, device="cpu")

# Step2: Computing the loss 
def compute_gradient(X, Y, R1, R2):
    '''
        the gradient of the loss with respect to the matrix encodes how much a tiny change 
    in some coordinate of that matrix affect the change of loss function.
        Gradient descent uses that information to iteratively change matrix R until we reach 
    a point where the loss is minimized.
    Inputs: 
        X: a matrix of dimension (m,n) where the colums are the contrast representation 
        Y: a matrix of dimension (m,n) where the colums are the reference representation
        R: a matrix of dimension (n,n) - transformation matrix from Y2d to X2d
    Outputs:
       g: a matrix of dimension (n,n) - gradient of the loss function L for given X, Y and R.
    '''
    # m is the number of rows in X
    m = len(X)

    rows, columns = X.shape

    gradient1 = (np.dot(X.T, np.dot(X, R1) - np.dot(Y, R2)) * 2)/rows
    gradient2 = (np.dot(Y.T, np.dot(Y, R2) - np.dot(X, R1)) * 2)/rows

    diff = np.dot(X, R1) - np.dot(Y, R2)

    # diff_squared is the element-wise square of the difference
    diff_squared = diff**2

    # sum_diff_squared is the sum of the squared elements
    sum_diff_squared = diff_squared.sum()

    # loss is the sum_diff_squared divided by the number of examples (m)
    loss = sum_diff_squared/m
    print("loss",loss)
    assert gradient1.shape == (columns, columns)
    assert gradient2.shape == (columns, columns)
    ### END CODE HERE ###
    return gradient1,gradient2

def cauculate(X,Y,R):
    m = len(X)
    Y_2d = projector.batch_project(200, Y)
    X_2d = projector.batch_project(200, np.dot(X, R))


    Euclid = 0
    for i in range(m):
        d = ((X_2d[i][0] - Y_2d[i][0])**2 + (X_2d[i][1] - Y_2d[i][1])**2)**0.5
        Euclid = Euclid + d
    return Euclid/m


def regression_loss(x, y):
    x = F.normalize(x, dim=1)
    y = F.normalize(y, dim=1)
    return 2 - 2 * (x * y).sum(dim=-1)
# Most of the time we iterate for a fixed number of training steps rather than iterating until the loss falls below a threshold.

# 1.Calculate gradient g of the loss with respect to the matrix R. 
# 2. Update R (Rnew = Rold - αg) . α is the learning rate which is a scalar.

# alignment_embeddings

def align_embeddings(X: np.ndarray, Y: np.ndarray,
                      learning_rate: float=0.0000000003,
                      seed: int=129) -> np.ndarray:
    '''
    Finding the optimal R with gradient descent algorithm
    Inputs:
        X: a matrix of dimension (m,n) where the colums are the contrast representation 
        Y: a matrix of dimension (m,n) where the colums are the reference representation
        learning_rate: positive float - describes how big steps will  gradient descent algorithm do.
    Outputs:
        R: a matrix of dimension (n,n) - the projection matrix that minimizes the F norm ||projector(X R) - projector ( Y )||^2
    '''
    # the number of columns in X is the number of dimensions for a word vector (e.g. 300)
    # R is a square matrix with length equal to the number of dimensions in th  word embedding
    # R1 = np.random.rand(X.shape[1], X.shape[1])
    # R2 = np.random.rand(X.shape[1], X.shape[1])
    m = len(X)
    R1 = Variable(torch.ones(X.shape[1],X.shape[1]),requires_grad=True)
    R2 = Variable(torch.ones(X.shape[1],X.shape[1]),requires_grad=True)
    # R = Variable(torch.ones(X.shape[1],X.shape[1]),requires_grad=True)
    X = torch.Tensor(X)
    Y = torch.Tensor(Y)
        
    train_steps = 6000
    for i in range(train_steps):

    
        loss = ((( X.matmul(R1).matmul(R2) - Y)**2).sum())/m
        if i%10 == 0:
            print(f"iteration {i} ===  loss: ",loss) 

        loss.backward()

        # lossCON = loss + lossFn
        # lossCON.backward()
        # grad = (np.dot(X.detach().numpy().T, np.dot(X.detach().numpy(), R.detach().numpy()) - Y.detach().numpy()) * 2)/rows

        R1.data = R1.data - learning_rate * R1.grad.data
        R2.data = R2.data - learning_rate * R2.grad.data
        ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
        # use the function that you defined to compute the gradient
        R1.grad.data.zero_()
        R2.grad.data.zero_()
        ### END CODE HERE ###
    
    return R1, R2

# np.random.seed(129)
# m = 10
# n = 5
X = confusion_data
Y = ref_train_data

# X_pred = exchange_provider.get_pred(200, ref_provider.train_representation(200))
# Y_pred = ref_provider.get_pred(200, ref_provider.train_representation(200))
R1, R2 = align_embeddings(X, Y)


print(R1)
import time
import json
filename = now = time.strftime("confusion_dataR1R2_ref_R1.json", time.localtime(time.time())) 
rlist = json.dumps(R1.tolist())
with open(filename, 'w', encoding='utf-8') as file_obj:

    file_obj.write(rlist)

filename2 = now = time.strftime("confusion_dataR1R2_ref_R2.json", time.localtime(time.time())) 
rlist = json.dumps(R2.tolist())
with open(filename2, 'w', encoding='utf-8') as file_obj:

    file_obj.write(rlist)

iteration 0 ===  loss:  tensor(2.5099e+13, grad_fn=<DivBackward0>)
iteration 10 ===  loss:  tensor(5.0802e+12, grad_fn=<DivBackward0>)
iteration 20 ===  loss:  tensor(2.1565e+12, grad_fn=<DivBackward0>)
iteration 30 ===  loss:  tensor(1.1921e+12, grad_fn=<DivBackward0>)
iteration 40 ===  loss:  tensor(7.5665e+11, grad_fn=<DivBackward0>)
iteration 50 ===  loss:  tensor(5.2316e+11, grad_fn=<DivBackward0>)
iteration 60 ===  loss:  tensor(3.8341e+11, grad_fn=<DivBackward0>)
iteration 70 ===  loss:  tensor(2.9312e+11, grad_fn=<DivBackward0>)
iteration 80 ===  loss:  tensor(2.3141e+11, grad_fn=<DivBackward0>)
iteration 90 ===  loss:  tensor(1.8735e+11, grad_fn=<DivBackward0>)
iteration 100 ===  loss:  tensor(1.5480e+11, grad_fn=<DivBackward0>)
iteration 110 ===  loss:  tensor(1.3006e+11, grad_fn=<DivBackward0>)
iteration 120 ===  loss:  tensor(1.1082e+11, grad_fn=<DivBackward0>)
iteration 130 ===  loss:  tensor(9.5560e+10, grad_fn=<DivBackward0>)
iteration 140 ===  loss:  tensor(8.3252e+10, 

KeyboardInterrupt: 